In [31]:
import pandas as pd
import plotly.express as px
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup
import seaborn as sns
import matplotlib.pyplot as plt
import os
import umap
import matplotlib.pyplot as plt
import glob
from top2vec import Top2Vec
data="./aylien_covid_news_cleaned_india"
full_df=pd.DataFrame()
for json_file in os.listdir(data):
    df=pd.read_json(os.path.join(data,json_file))
    full_df=pd.concat([full_df,df])

In [2]:
full_df=full_df.sort_values(by=["publish_time"])

In [3]:
weekly_df=pd.DataFrame()
news=[]
sentiment=[]
timeline=[]
for week in full_df['publish_time'].dt.week.unique():
    this_weeks_data = full_df[full_df['publish_time'].dt.week == week]
    news.append([this_weeks_data.summary.values[i] for i in range(len(this_weeks_data.summary.values))])
    sentiment.append([i for i in this_weeks_data.body_polarity_score.values])
    timeline.append(str(this_weeks_data.publish_time.iloc[0])+"-"+str(this_weeks_data.publish_time.iloc[-1]))
weekly_df["week_timespan"]=timeline
weekly_df["week_news"]=news
weekly_df["sentiment"]=[len([j for j in ast.literal_eval(i) if j<0.4]) for i in news.sentiment.values]


In [42]:
weekly_df.index=pd.to_datetime(pd.Series([i[:11] for i in weekly_df.week_timespan.values]))

In [44]:
weekly_df

,week_timespan,week_news,sentiment,Emotions,down,up
2019-11-01,2019-11-01 06:45:40-2019-11-01 06:51:38,[NEW DELHI: Bats in some parts of Nagaland may...,"[0.647077, 0.824676]","[1, 1]",0,1
2020-01-09,2020-01-09 04:15:42-2020-01-12 00:33:41,[China believes a mysterious outbreak that has...,"[0.942979, 0.8793489999999999, 0.732157, 0.963...","[1, 1, 1, 1, 1, 1, 4, 1]",0,5
2020-01-13,2020-01-13 14:43:38-2020-01-19 20:43:50,[The World Health Organisation (WHO) has cauti...,"[0.930164, 0.783308, 0.847429, 0.500248, 0.717...","[1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, ...",2,26
2020-01-20,2020-01-20 00:50:37-2020-01-26 22:47:02,[China on Monday reported a mysterious SARS-li...,"[0.888312, 0.867571, 0.9308369999999999, 0.916...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, ...",20,217
2020-01-27,2020-01-27 00:30:31-2020-02-02 23:48:28,[China’s Central government said on Monday tha...,"[0.512922, 0.48407599999999995, 0.692087999999...","[2, 1, 4, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, ...",26,607
2020-02-03,2020-02-03 00:15:16-2020-02-09 23:59:28,[A testing facility for the detection of novel...,"[0.563288, 0.966329, 0.9368409999999999, 0.448...","[1, 2, 1, 1, 2, 2, 1, 0, 2, 1, 1, 2, 2, 1, 0, ...",46,590
2020-02-10,2020-02-10 00:32:59-2020-02-16 23:40:55,"[Bhubaneswar: Family members of two youths, wh...","[0.746371, 0.651656, 0.8182419999999999, 0.711...","[4, 1, 1, 1, 1, 2, 1, 4, 0, 2, 2, 4, 1, 1, 1, ...",17,478
2020-02-17,2020-02-17 00:41:58-2020-02-23 23:48:12,[Singer Rewben Mashangva performs the song at ...,"[0.936708, 0.756846, 0.550484, 0.469887, 0.739...","[2, 4, 4, 0, 4, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, ...",27,410
2020-02-24,2020-02-24 00:51:54-2020-03-01 23:47:58,"[The health chief of the Lombardy region, Giul...","[0.564616, 0.9019919999999999, 0.9988849999999...","[4, 2, 1, 2, 0, 1, 2, 2, 4, 1, 2, 1, 2, 2, 1, ...",36,605
2020-03-02,2020-03-02 00:31:35-2020-03-08 23:59:40,[The UN health agency stressed in its latest s...,"[0.9261889999999999, 0.9722919999999999, 0.420...","[4, 2, 1, 2, 4, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, ...",59,1745


# Tried Using NRC Emotion Lexicon Dataset to map the emotion 

In [6]:
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer

In [7]:
filepath = ('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
emotion_lex = pd.read_csv(filepath,
                            names=["word", "emotion", "association"],
                            sep='\t')

In [8]:
emotion_lex

,word,emotion,association
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0
...,...,...,...
141815,zoom,negative,0
141816,zoom,positive,0
141817,zoom,sadness,0
141818,zoom,surprise,0


In [90]:
text="someone is running behind me to kill me"
tokens=word_tokenize(text)
stemmer=SnowballStemmer("english")

In [93]:
stemmed=[stemmer.stem(i) for i in tokens]

In [113]:
for i in stemmed:
    print(emotion_lex[emotion_lex.word==i][emotion_lex.association==1].emotion.values)
    

[]
[]
[]
[]
[]
[]
['fear' 'negative' 'sadness']
[]


C:\Users\akshe\Miniconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


# Trained Emotinal Bert .Lets play with models not emotions

In [5]:
model=torch.load("emotion_sentiment.pt").to("cuda:0")

C:\Users\akshe\Miniconda3\lib\site-packages\torch\serialization.py:593: SourceChangeWarning: source code of class 'transformers.modeling_bert.BertSelfAttention' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
def emotion_classifier(text):
    test_data=tokenizer.encode_plus(
      text,
      max_length=32,
      add_special_tokens=True,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    pred=model(test_data["input_ids"].to("cuda:0"),test_data["attention_mask"].to("cuda:0"))[0].to("cpu").detach().numpy()
    return np.argmax(pred)
emotion_classifier(weekly_df.week_news[10][0])

2

In [15]:
labels_mapping={'sadness': 4, 'joy': 2, 'anger': 0, 'fear': 1, 'surprise': 5}

In [22]:
news_emotions=[[emotion_classifier(j) for j in i] for i in weekly_df.week_news.values]

In [24]:
len(news_emotions)

22

In [25]:
weekly_df["Emotions"]=news_emotions

In [47]:
weekly_df["down_sentiment"]=[len([j for j in i if j<0.4]) for i in sentiment]
weekly_df["up_sentiment"]=[len([j for j in i if j>0.8]) for i in sentiment]

In [56]:
def emotion_postproc(news_emotions,emotion):
    number_of_emotion_week=[]
    for i in news_emotions:
        emotion_count_this_week=0
        for j in i:
            if j==emotion:
                emotion_count_this_week+=1
        number_of_emotion_week.append((emotion_count_this_week/len(i))*100)
    return number_of_emotion_week

In [57]:
weekly_df["fear"]=emotion_postproc(news_emotions,1)
weekly_df["anger"]=emotion_postproc(news_emotions,0)
weekly_df["sadness"]=emotion_postproc(news_emotions,4)
weekly_df["joy"]=emotion_postproc(news_emotions,2)
weekly_df["surprise"]=emotion_postproc(news_emotions,5)

In [59]:
weekly_df.to_csv("News_weekly_cleaned.csv")

In [58]:
weekly_df

,week_timespan,week_news,sentiment,Emotions,down,up,down_sentiment,up_sentiment,fear,anger,sadness,joy,surprise
2019-11-01,2019-11-01 06:45:40-2019-11-01 06:51:38,[NEW DELHI: Bats in some parts of Nagaland may...,"[0.647077, 0.824676]","[1, 1]",0,1,0,1,100.000000,0.000000,0.000000,0.000000,0.000000
2020-01-09,2020-01-09 04:15:42-2020-01-12 00:33:41,[China believes a mysterious outbreak that has...,"[0.942979, 0.8793489999999999, 0.732157, 0.963...","[1, 1, 1, 1, 1, 1, 4, 1]",0,5,0,5,87.500000,0.000000,12.500000,0.000000,0.000000
2020-01-13,2020-01-13 14:43:38-2020-01-19 20:43:50,[The World Health Organisation (WHO) has cauti...,"[0.930164, 0.783308, 0.847429, 0.500248, 0.717...","[1, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, ...",2,26,2,26,86.792453,1.886792,1.886792,9.433962,0.000000
2020-01-20,2020-01-20 00:50:37-2020-01-26 22:47:02,[China on Monday reported a mysterious SARS-li...,"[0.888312, 0.867571, 0.9308369999999999, 0.916...","[1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, ...",20,217,20,217,59.640523,13.888889,6.209150,20.261438,0.000000
2020-01-27,2020-01-27 00:30:31-2020-02-02 23:48:28,[China’s Central government said on Monday tha...,"[0.512922, 0.48407599999999995, 0.692087999999...","[2, 1, 4, 1, 2, 1, 1, 2, 1, 1, 1, 0, 1, 1, 0, ...",26,607,26,607,55.398530,16.506501,6.218202,21.876767,0.000000
2020-02-03,2020-02-03 00:15:16-2020-02-09 23:59:28,[A testing facility for the detection of novel...,"[0.563288, 0.966329, 0.9368409999999999, 0.448...","[1, 2, 1, 1, 2, 2, 1, 0, 2, 1, 1, 2, 2, 1, 0, ...",46,590,46,590,45.883055,21.360382,7.875895,24.761337,0.059666
2020-02-10,2020-02-10 00:32:59-2020-02-16 23:40:55,"[Bhubaneswar: Family members of two youths, wh...","[0.746371, 0.651656, 0.8182419999999999, 0.711...","[4, 1, 1, 1, 1, 2, 1, 4, 0, 2, 2, 4, 1, 1, 1, ...",17,478,17,478,38.656716,17.164179,11.119403,32.985075,0.000000
2020-02-17,2020-02-17 00:41:58-2020-02-23 23:48:12,[Singer Rewben Mashangva performs the song at ...,"[0.936708, 0.756846, 0.550484, 0.469887, 0.739...","[2, 4, 4, 0, 4, 1, 0, 2, 1, 0, 1, 2, 1, 0, 1, ...",27,410,27,410,36.256324,20.994941,10.118044,32.293423,0.084317
2020-02-24,2020-02-24 00:51:54-2020-03-01 23:47:58,"[The health chief of the Lombardy region, Giul...","[0.564616, 0.9019919999999999, 0.9988849999999...","[4, 2, 1, 2, 0, 1, 2, 2, 4, 1, 2, 1, 2, 2, 1, ...",36,605,36,605,33.785450,21.578298,10.604192,33.785450,0.000000
2020-03-02,2020-03-02 00:31:35-2020-03-08 23:59:40,[The UN health agency stressed in its latest s...,"[0.9261889999999999, 0.9722919999999999, 0.420...","[4, 2, 1, 2, 4, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, ...",59,1745,59,1745,40.451268,23.563619,6.862061,28.867178,0.093045
